# 2-3: Evidently 드리프트 모니터링

## 학습 목표
- **드리프트(Drift)** 개념 이해
- Evidently로 데이터/타겟 드리프트 감지
- FDS에서 드리프트가 왜 중요한지 학습
- 자동화된 모니터링 (TestSuite) 구축

## 이 노트북에서 할 것
```
학습 데이터 (Reference)     최근 데이터 (Current)
        ↓                         ↓
              Evidently 비교
                   ↓
         드리프트 감지 리포트
                   ↓
         PSI > 0.25? → 재학습 알림!
```

## 사전 요구사항
- `pip install evidently`
- 2-1, 2-2 노트북 실행 완료

## Evidently API 버전 안내
- **이 노트북은 Evidently 0.6.8+ (신규 API) 기준**
- 구버전(≤0.6.7)은 `ColumnMapping` 사용, 신버전은 `DataDefinition` 사용

## 예상 시간: 약 2시간

In [2]:
# 패키지 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Evidently 핵심 모듈 (신규 API - 0.6.8+)
import evidently
from evidently import Report
from evidently.presets import DataDriftPreset

# 한글 폰트
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 경로 설정
DATA_PROCESSED = Path('../../data/processed')
REPORTS_DIR = Path('../../reports')
REPORTS_DIR.mkdir(exist_ok=True)  # reports 폴더 생성

print(f"Evidently 버전: {evidently.__version__}")
print("패키지 로드 완료!")
print(f"리포트 저장 경로: {REPORTS_DIR.absolute()}")

Evidently 버전: 0.7.19
패키지 로드 완료!
리포트 저장 경로: C:\workspace\fds-system\notebooks\phase2\..\..\reports


---

## Part 1: 드리프트 개념

### 1-1. 드리프트(Drift)란?

**드리프트** = 시간이 지나면서 데이터 분포가 변하는 현상

```
[모델 학습 시점]              [3개월 후 운영 중]

거래금액 분포:                거래금액 분포:
    ▓▓▓▓                         ▓▓
   ▓▓▓▓▓▓                       ▓▓▓▓▓▓▓▓
  ▓▓▓▓▓▓▓▓                     ▓▓▓▓▓▓▓▓▓▓
 ▓▓▓▓▓▓▓▓▓▓                   ▓▓▓▓▓▓▓▓▓▓▓▓
─────────────               ─────────────────
  평균: 10만원                  평균: 25만원 ← 분포가 바뀜!
```

**왜 문제인가?**
- 모델은 학습 데이터의 패턴을 배움
- 실제 데이터가 달라지면 예측 성능 저하
- FDS: 사기 탐지율(Recall) 급락 가능!

### 1-2. 드리프트 3가지 종류

| 종류 | 영어 | 설명 | FDS 예시 |
|------|------|------|----------|
| **데이터 드리프트** | Data Drift | 입력 피처 분포 변화 | 거래금액 평균 증가, 야간 거래 증가 |
| **타겟 드리프트** | Target Drift | 정답(라벨) 분포 변화 | 사기 비율 3.5% → 7%로 증가 |
| **개념 드리프트** | Concept Drift | 입력-출력 관계 변화 | 같은 패턴이 이제는 사기가 아님 |

```
Data Drift:    X(피처)의 분포 변화
Target Drift:  Y(정답)의 분포 변화
Concept Drift: P(Y|X) 관계 자체가 변화
```

**이 노트북에서 다루는 것:**
- Part 3: 데이터 드리프트 (Data Drift) ⭐
- Part 4: 타겟 드리프트 (Target Drift)

### 1-3. FDS에서 왜 중요한가?

**실제 시나리오:**

```
1월: 모델 배포 (Recall 90%)
          ↓
3월: 새로운 사기 수법 등장 (딥페이크 활용)
          ↓
4월: 기존 피처로는 탐지 불가
          ↓
5월: Recall 60%로 급락 😱
          ↓
     손실 발생 후에야 문제 인지
```

**드리프트 모니터링 적용 시:**

```
1월: 모델 배포 + 드리프트 모니터링 시작
          ↓
3월: 새로운 사기 수법 등장
          ↓
3월 2주차: PSI > 0.25 감지! 알림 발송 🚨
          ↓
3월 3주차: 원인 분석 → 새 피처 추가 → 재학습
          ↓
4월: 새 모델 배포 (Recall 88% 유지) ✅
```

### 1-4. Evidently 소개

**Evidently** = 오픈소스 ML 모니터링 프레임워크

```
Evidently 주요 기능:
├── Data Drift 감지
├── Target Drift 감지
├── 모델 성능 모니터링
├── 데이터 품질 체크
└── HTML 리포트 생성
```

**왜 Evidently를 선택했나?**

| 특징 | 설명 |
|------|------|
| 무료 오픈소스 | 상용 도구 대비 비용 절감 |
| 쉬운 사용법 | 5줄 코드로 드리프트 감지 |
| 시각화 | HTML 리포트 자동 생성 |
| 자동화 | TestSuite로 CI/CD 연동 |
| JD 빈도 55% | 취업 시장에서 많이 요구 |

In [9]:
# 실습 1: Evidently 버전 및 API 확인

print(f"Evidently 버전: {evidently.__version__}")

# 버전에 따른 API 안내
version_parts = evidently.__version__.split('.')
major, minor = int(version_parts[0]), int(version_parts[1])

if major == 0 and minor >= 6:
    print("\n✅ 신규 API 사용 (0.6.8+)")
    print("   - DataDefinition 대신 자동 컬럼 감지")
    print("   - Report([preset]).run(reference, current)")
else:
    print("\n⚠️ 구버전 API")
    print("   - ColumnMapping 사용")

print("\n사용 가능한 Preset:")
print("  - DataDriftPreset: 데이터 드리프트 분석")
print("  - DataQualityPreset: 데이터 품질 분석")

print("\n체크포인트 1 통과: Evidently 설치 확인!")

Evidently 버전: 0.7.19

✅ 신규 API 사용 (0.6.8+)
   - DataDefinition 대신 자동 컬럼 감지
   - Report([preset]).run(reference, current)

사용 가능한 Preset:
  - DataDriftPreset: 데이터 드리프트 분석
  - DataQualityPreset: 데이터 품질 분석

체크포인트 1 통과: Evidently 설치 확인!


---

## Part 2: 데이터 준비

### 2-1. Reference vs Current 개념

Evidently는 **두 데이터셋을 비교**해서 드리프트를 감지합니다.

```
┌─────────────────┐      ┌─────────────────┐
│  Reference Data │      │  Current Data   │
│   (기준 데이터)  │  vs  │  (현재 데이터)   │
│                 │      │                 │
│  = 학습 데이터   │      │  = 최근 데이터   │
│  = "정상" 기준   │      │  = 비교 대상     │
└─────────────────┘      └─────────────────┘
          │                      │
          └──────────┬───────────┘
                     ↓
              분포 비교 (PSI 등)
                     ↓
              드리프트 여부 판정
```

| 파라미터 | 역할 | FDS 예시 |
|----------|------|----------|
| `reference_data` | 기준 (정상) 데이터 | 모델 학습에 사용한 데이터 |
| `current_data` | 비교할 데이터 | 최근 1주일 거래 데이터 |

In [4]:
# 예제: 데이터 로드

# 학습 데이터 (Reference)
train_df = pd.read_csv(DATA_PROCESSED / 'train_features.csv')

# 테스트 데이터 (Current로 사용)
test_df = pd.read_csv(DATA_PROCESSED / 'test_features.csv')

print(f"Reference 데이터 (train): {train_df.shape}")
print(f"Current 데이터 (test): {test_df.shape}")
print(f"\n피처 수: {train_df.shape[1] - 1}개")  # isFraud 제외
print(f"\nReference 사기 비율: {train_df['isFraud'].mean():.2%}")
print(f"Current 사기 비율: {test_df['isFraud'].mean():.2%}")

Reference 데이터 (train): (472432, 448)
Current 데이터 (test): (118108, 448)

피처 수: 447개

Reference 사기 비율: 3.51%
Current 사기 비율: 3.44%


### 2-2. 드리프트 시뮬레이션 데이터 생성

실제 드리프트를 확인하기 위해, **인위적으로 드리프트가 있는 데이터**를 만들어봅니다.

```
시뮬레이션 시나리오:
- 거래금액(TransactionAmt) 평균 1.5배 증가
- 일부 피처 분포 변경
- 사기 비율 증가
```

In [10]:
# 예제: 드리프트 시뮬레이션 데이터 생성

def create_drifted_data(df, drift_strength=0.3, seed=42):
    """
    드리프트가 있는 시뮬레이션 데이터 생성
    
    Parameters:
    -----------
    df : pd.DataFrame
        원본 데이터
    drift_strength : float
        드리프트 강도 (0.0 ~ 1.0)
        - 0.0: 드리프트 없음
        - 0.3: 약한 드리프트
        - 0.7: 강한 드리프트
    seed : int
        랜덤 시드 (재현성)
    
    Returns:
    --------
    drifted_df : pd.DataFrame
        드리프트된 데이터
    drift_cols : list
        드리프트가 적용된 컬럼 목록
    """
    np.random.seed(seed)
    drifted_df = df.copy()
    
    # 숫자형 컬럼 찾기 (isFraud 제외)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'isFraud' in numeric_cols:
        numeric_cols.remove('isFraud')
    
    # 드리프트 적용할 피처 선택 (상위 10개)
    drift_cols = numeric_cols[:10]
    
    for col in drift_cols:
        # 평균과 표준편차 기반 드리프트
        col_mean = df[col].mean()
        col_std = df[col].std()
        
        if col_std > 0:  # 표준편차가 0이 아닌 경우만
            # 드리프트: 평균 이동 + 노이즈 추가
            shift = col_mean * drift_strength
            noise = np.random.normal(0, col_std * drift_strength * 0.5, len(df))
            drifted_df[col] = df[col] + shift + noise
    
    return drifted_df, drift_cols

# 드리프트 데이터 생성
drifted_df, drifted_columns = create_drifted_data(test_df, drift_strength=0.3)

print("드리프트 시뮬레이션 데이터 생성 완료!")
print(f"\n드리프트 적용된 피처 ({len(drifted_columns)}개):")
for col in drifted_columns[:5]:
    orig_mean = test_df[col].mean()
    drift_mean = drifted_df[col].mean()
    if orig_mean != 0:
        change = (drift_mean - orig_mean) / abs(orig_mean) * 100
    else:
        change = 0
    print(f"  - {col}: {orig_mean:.2f} → {drift_mean:.2f} ({change:+.1f}%)")

드리프트 시뮬레이션 데이터 생성 완료!

드리프트 적용된 피처 (10개):
  - TransactionAmt: 137.53 → 178.88 (+30.1%)
  - ProductCD: 3.38 → 4.40 (+30.0%)
  - card1: 10022.20 → 13025.52 (+30.0%)
  - card2: 361.33 → 469.68 (+30.0%)
  - card3: 152.74 → 198.55 (+30.0%)


In [11]:
# 실습 2: 데이터 준비 확인

# Reference 데이터 확인
print(f"Reference 데이터 크기: {train_df.shape}")

# Current 데이터 확인 (드리프트된 데이터)
print(f"Current 데이터 크기 (드리프트): {drifted_df.shape}")
print(f"Current 데이터 크기 (원본): {test_df.shape}")

# 드리프트 전후 비교
if drifted_columns:
    first_col = drifted_columns[0]
    print(f"\n{first_col} 평균 비교:")
    print(f"  Reference: {train_df[first_col].mean():.4f}")
    print(f"  Current (원본): {test_df[first_col].mean():.4f}")
    print(f"  Current (드리프트): {drifted_df[first_col].mean():.4f}")

Reference 데이터 크기: (472432, 448)
Current 데이터 크기 (드리프트): (118108, 448)
Current 데이터 크기 (원본): (118108, 448)

TransactionAmt 평균 비교:
  Reference: 134.4014
  Current (원본): 137.5303
  Current (드리프트): 178.8773


In [12]:
# 체크포인트 2: 데이터 준비 확인
assert train_df is not None, "Reference 데이터가 없습니다!"
assert drifted_df is not None, "Current 데이터가 없습니다!"
assert len(drifted_columns) > 0, "드리프트 컬럼이 없습니다!"
print("체크포인트 2 통과: 데이터 준비 완료!")

체크포인트 2 통과: 데이터 준비 완료!


---

## Part 3: 데이터 드리프트 분석 ⭐

### 3-1. DataDriftPreset 사용법

**DataDriftPreset** = 데이터 드리프트 분석을 위한 미리 정의된 설정

```python
# Evidently 0.6.8+ 신규 API
from evidently import Report
from evidently.presets import DataDriftPreset

# 리포트 생성 및 실행
report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=train_df, current_data=test_df)
```

**주요 파라미터:**

| 파라미터 | 설명 | 기본값 |
|----------|------|--------|
| `reference_data` | 기준 데이터 (DataFrame) | 필수 |
| `current_data` | 비교 데이터 (DataFrame) | 필수 |

**Evidently가 자동으로 하는 것:**
- 컬럼 타입 자동 감지 (숫자형/범주형)
- 적절한 통계 검정 방법 선택
- 드리프트 여부 판정

### 3-2. PSI (Population Stability Index) 이해

**PSI란?**
- 두 분포가 **얼마나 다른지** 측정하는 지표
- 금융권에서 가장 많이 사용하는 표준 지표

**PSI 계산 방법 (참고용):**
```
PSI = Σ (현재비율 - 기준비율) × ln(현재비율/기준비율)

각 구간(bin)별로 계산 후 합산
```

**PSI 해석 기준 (금융권 표준):**

| PSI 값 | 해석 | 조치 |
|--------|------|------|
| **< 0.1** | 변화 없음 (안정) | 유지 |
| **0.1 ~ 0.25** | 약간 변화 | 모니터링 강화 ⚠️ |
| **> 0.25** | 큰 변화 (드리프트) | **재학습 검토** 🚨 |

```
예시:
- 거래금액 PSI = 0.05 → 안정, 문제없음
- 거래금액 PSI = 0.15 → 약간 변화, 주시 필요
- 거래금액 PSI = 0.35 → 드리프트! 재학습 검토
```

In [13]:
%%time
# 예제: 데이터 드리프트 리포트 생성 (드리프트 있는 데이터)

# 1. Report 객체 생성
#    - metrics: 어떤 분석을 할지 지정
#    - DataDriftPreset: 데이터 드리프트 분석 프리셋
drift_report = Report(metrics=[
    DataDriftPreset()  # 자동으로 적절한 통계 검정 선택
])

# 2. 리포트 실행 (신규 API)
#    - reference_data: 기준 데이터 (학습 데이터)
#    - current_data: 비교할 데이터 (드리프트 시뮬레이션)
#    ⭐ 중요: run()은 결과 객체를 반환! 이 객체에서 .dict(), .save_html() 등 호출
drift_result = drift_report.run(
    reference_data=train_df,
    current_data=drifted_df
)

print("데이터 드리프트 리포트 생성 완료! (드리프트 있는 데이터)")

데이터 드리프트 리포트 생성 완료! (드리프트 있는 데이터)
CPU times: total: 5min 18s
Wall time: 5min 18s


In [14]:
# 예제: 리포트 시각화 (Jupyter에서 바로 보기)
# 인터랙티브 리포트가 표시됩니다
# ⭐ run() 결과 객체를 표시
drift_result

[HTML 리포트 출력 생략 - reports/ 폴더 참조]


### 3-3. 드리프트 결과 프로그래밍 방식으로 확인

리포트 결과를 **코드로 추출**하여 자동화에 활용할 수 있습니다.

In [15]:
# 예제: 드리프트 결과 추출 (Evidently 0.7+ 호환)

# 결과를 딕셔너리로 변환
result_dict = drift_result.dict()

print("=== 데이터 드리프트 분석 결과 ===")

# Evidently 0.7+ 구조: metrics 배열에서 metric_name, value 사용
metrics_results = result_dict.get('metrics', [])
print(f"\n메트릭 수: {len(metrics_results)}")

# 드리프트 요약 찾기 (DriftedColumnsCount)
drift_summary = None
for metric in metrics_results:
    metric_name = str(metric.get('metric_name', ''))
    if 'DriftedColumnsCount' in metric_name:
        drift_summary = metric.get('value', {})
        print(f"\n[요약 메트릭] {metric_name}")
        break

if drift_summary and isinstance(drift_summary, dict):
    drift_count = drift_summary.get('count', 0)
    drift_share = drift_summary.get('share', 0)
    
    print(f"드리프트된 피처 수: {int(drift_count)}")
    print(f"드리프트된 피처 비율: {drift_share:.1%}")
    
    # 드리프트 여부 판단 (share > 0.3이면 드리프트)
    dataset_drift = drift_share > 0.3
    print(f"\n전체 데이터셋 드리프트: {'예 🚨' if dataset_drift else '아니오 ✅'}")
else:
    print("드리프트 요약 정보를 찾을 수 없습니다.")
    print("HTML 리포트에서 확인하세요.")

=== 데이터 드리프트 분석 결과 ===

메트릭 수: 449

[요약 메트릭] DriftedColumnsCount(drift_share=0.5)
드리프트된 피처 수: 147
드리프트된 피처 비율: 32.8%

전체 데이터셋 드리프트: 예 🚨


In [16]:
# 예제: 개별 피처 드리프트 상세 확인 (Evidently 0.7+ 호환)
import re

# 드리프트 점수가 높은 피처 찾기
drift_scores = []

for metric in metrics_results:
    metric_name = str(metric.get('metric_name', ''))
    
    # ValueDrift 메트릭에서 컬럼별 드리프트 정보 추출
    # 예: "ValueDrift(column=TransactionAmt,method=K-S p_value,threshold=0.05)"
    if 'ValueDrift' in metric_name:
        col_match = re.search(r'column=([^,)]+)', metric_name)
        threshold_match = re.search(r'threshold=([0-9.]+)', metric_name)
        
        if col_match:
            col_name = col_match.group(1)
            p_value = metric.get('value', 1.0)  # p-value
            threshold = float(threshold_match.group(1)) if threshold_match else 0.05
            
            # p-value < threshold → 드리프트 감지
            drift_detected = p_value < threshold
            
            drift_scores.append({
                'column': col_name,
                'drift_detected': drift_detected,
                'drift_score': p_value,  # p-value (낮을수록 드리프트)
                'stattest_name': 'K-S p_value'
            })

# DataFrame으로 변환 및 정렬 (p-value 낮은 순 = 드리프트 심한 순)
if drift_scores:
    drift_df = pd.DataFrame(drift_scores)
    drift_df = drift_df.sort_values('drift_score', ascending=True)  # p-value 낮은 순
    
    print("=== 드리프트 점수 상위 10개 피처 (p-value 낮은 순) ===")
    print(drift_df.head(10).to_string(index=False))
    
    # 드리프트 감지된 피처만
    drifted_features = drift_df[drift_df['drift_detected'] == True]
    print(f"\n드리프트 감지된 피처 수: {len(drifted_features)}개")
else:
    print("드리프트 점수 정보를 찾을 수 없습니다.")
    drift_df = pd.DataFrame()

=== 드리프트 점수 상위 10개 피처 (p-value 낮은 순) ===
 column  drift_detected  drift_score stattest_name
    V65            True     0.000087   K-S p_value
    V14            True     0.000107   K-S p_value
isFraud            True     0.001401   K-S p_value
   V305            True     0.001713   K-S p_value
   V241            True     0.001724   K-S p_value
    V41            True     0.001798   K-S p_value
    V71            True     0.001897   K-S p_value
     V1            True     0.002020   K-S p_value
    V88            True     0.002407   K-S p_value
    V27            True     0.002986   K-S p_value

드리프트 감지된 피처 수: 301개


In [17]:
# 실습 3: 드리프트 없는 데이터로 비교

# 드리프트 없는 데이터 (원본 test_df)로 리포트 생성
no_drift_report = Report(metrics=[
    DataDriftPreset()
])

# reference와 current 모두 비슷한 분포의 데이터 사용
# ⭐ run() 결과를 변수에 저장
no_drift_result = no_drift_report.run(
    reference_data=train_df,
    current_data=test_df  # 드리프트 없는 원본 데이터
)

print("드리프트 없는 데이터 리포트 생성 완료!")
print("\n아래 리포트와 위의 드리프트 리포트를 비교해보세요:")

드리프트 없는 데이터 리포트 생성 완료!

아래 리포트와 위의 드리프트 리포트를 비교해보세요:


In [18]:
# 드리프트 없는 리포트 시각화
# ⭐ run() 결과 객체를 표시
no_drift_result

[HTML 리포트 출력 생략 - reports/ 폴더 참조]


In [19]:
# 체크포인트 3: 데이터 드리프트 분석 확인
assert drift_result is not None, "드리프트 리포트가 생성되지 않았습니다!"
assert no_drift_result is not None, "비교 리포트가 생성되지 않았습니다!"
print("체크포인트 3 통과: 데이터 드리프트 분석 완료!")

체크포인트 3 통과: 데이터 드리프트 분석 완료!


---

## Part 4: 타겟 드리프트

### 4-1. 타겟 드리프트란?

**타겟 드리프트** = 정답(라벨) 분포의 변화

```
FDS 예시:
- Reference: 사기 비율 3.5%
- Current: 사기 비율 7.0%  ← 2배 증가!

원인 가능성:
1. 실제로 사기가 증가함 → 모델 재학습 필요
2. 데이터 수집 방식 변경 → 데이터 파이프라인 점검
3. 라벨링 기준 변경 → 비즈니스 확인 필요
```

In [20]:
# 예제: 타겟 드리프트 시뮬레이션

def create_target_drift(df, new_fraud_rate=0.07, seed=42):
    """
    타겟(사기 비율) 드리프트 시뮬레이션
    
    Parameters:
    -----------
    df : pd.DataFrame
        원본 데이터
    new_fraud_rate : float
        새로운 사기 비율 (0.07 = 7%)
    seed : int
        랜덤 시드
    
    Returns:
    --------
    target_drifted_df : pd.DataFrame
        타겟 드리프트된 데이터
    """
    np.random.seed(seed)
    target_drifted_df = df.copy()
    
    n_samples = len(df)
    n_fraud_needed = int(n_samples * new_fraud_rate)
    n_fraud_current = int(df['isFraud'].sum())
    
    if n_fraud_needed > n_fraud_current:
        # 사기 비율 증가: 일부 정상을 사기로 변경
        normal_idx = df[df['isFraud'] == 0].index
        flip_count = min(n_fraud_needed - n_fraud_current, len(normal_idx))
        flip_idx = np.random.choice(normal_idx, size=int(flip_count), replace=False)
        target_drifted_df.loc[flip_idx, 'isFraud'] = 1
    
    return target_drifted_df

# 타겟 드리프트 데이터 생성 (사기 비율 → 7%)
target_drifted_df = create_target_drift(test_df, new_fraud_rate=0.07)

print("타겟 드리프트 시뮬레이션 완료!")
print(f"\nReference 사기 비율: {train_df['isFraud'].mean():.2%}")
print(f"Current 사기 비율 (원본): {test_df['isFraud'].mean():.2%}")
print(f"Current 사기 비율 (드리프트): {target_drifted_df['isFraud'].mean():.2%}")

타겟 드리프트 시뮬레이션 완료!

Reference 사기 비율: 3.51%
Current 사기 비율 (원본): 3.44%
Current 사기 비율 (드리프트): 7.00%


In [21]:
# 예제: 타겟 컬럼만 비교하는 간단한 분석

# isFraud 컬럼만 추출해서 비교
target_ref = train_df[['isFraud']].copy()
target_cur = target_drifted_df[['isFraud']].copy()

# 타겟 드리프트 리포트
target_drift_report = Report(metrics=[
    DataDriftPreset()  # 타겟 컬럼에 대한 드리프트 분석
])

# ⭐ run() 결과를 변수에 저장
target_drift_result = target_drift_report.run(
    reference_data=target_ref,
    current_data=target_cur
)

print("타겟 드리프트 리포트 생성 완료!")

타겟 드리프트 리포트 생성 완료!


In [22]:
# 타겟 드리프트 리포트 시각화
# ⭐ run() 결과 객체를 표시
target_drift_result

[HTML 리포트 출력 생략 - reports/ 폴더 참조]


In [23]:
# 실습 4: 타겟 드리프트 결과 확인

# 타겟 드리프트 결과 추출
# ⭐ run() 반환값에서 .dict() 호출
target_result = target_drift_result.dict()

print("=== 타겟 드리프트 분석 결과 ===")
print(f"\nReference 사기 비율: {train_df['isFraud'].mean():.2%}")
print(f"Current 사기 비율: {target_drifted_df['isFraud'].mean():.2%}")

# 비율 변화 계산
ref_rate = train_df['isFraud'].mean()
cur_rate = target_drifted_df['isFraud'].mean()
change_pct = (cur_rate - ref_rate) / ref_rate * 100 if ref_rate > 0 else 0
print(f"변화율: {change_pct:+.1f}%")

# 결과에서 드리프트 여부 확인
for metric in target_result.get('metrics', []):
    result = metric.get('result', {})
    if 'dataset_drift' in result:
        print(f"\n타겟 드리프트 감지: {'예 🚨' if result['dataset_drift'] else '아니오 ✅'}")

=== 타겟 드리프트 분석 결과 ===

Reference 사기 비율: 3.51%
Current 사기 비율: 7.00%
변화율: +99.2%


In [24]:
# 체크포인트 4: 타겟 드리프트 분석 확인
assert target_drift_result is not None, "타겟 드리프트 리포트가 생성되지 않았습니다!"
print("체크포인트 4 통과: 타겟 드리프트 분석 완료!")

체크포인트 4 통과: 타겟 드리프트 분석 완료!


---

## Part 5: 자동화 테스트 (include_tests)

### 5-1. Report + Tests 개념 (Evidently 0.7+)

**Evidently 0.7.0부터 TestSuite가 Report에 통합**되었습니다.

```python
# 구버전 (0.6.x) - deprecated
from evidently.test_suite import TestSuite  # ❌ 더 이상 안됨

# 신버전 (0.7+) - Report에 테스트 포함
report = Report(
    metrics=[DataDriftPreset()],
    include_tests=True  # ✅ 테스트 자동 포함
)
```

**Report + Tests의 장점:**

| 구분 | 설명 |
|------|------|
| **간단한 API** | Report 하나로 분석 + 테스트 |
| **자동 조건** | reference 데이터 기반 자동 임계값 |
| **CI/CD 연동** | Pass/Fail 결과로 자동화 가능 |

```
실무 워크플로:
1. include_tests=True로 자동 테스트 (매일/매주)
2. 실패 시 알림 발송
3. 상세 리포트로 원인 분석
4. 조치 (재학습 등)
```

In [25]:
# 예제: Report + Tests 사용법 (Evidently 0.7+)

# 테스트가 포함된 리포트 생성
# include_tests=True: reference 데이터 기반으로 자동 테스트 조건 생성
drift_test_report = Report(
    metrics=[DataDriftPreset()],
    include_tests=True  # ⭐ 테스트 자동 포함!
)

# 드리프트 있는 데이터로 테스트
drift_test_result = drift_test_report.run(
    reference_data=train_df,
    current_data=drifted_df
)

print("Report + Tests 실행 완료! (드리프트 있는 데이터)")

Report + Tests 실행 완료! (드리프트 있는 데이터)


In [26]:
# 테스트 결과 시각화
# ⭐ 리포트에 테스트 결과도 함께 표시됨
drift_test_result

[HTML 리포트 출력 생략 - reports/ 폴더 참조]


In [27]:
# 예제: 테스트 결과 프로그래밍 방식으로 확인

# 결과를 딕셔너리로 변환
test_results = drift_test_result.dict()

print("=== 테스트 결과 ===")

# tests 결과 확인
tests_info = test_results.get('tests', [])
if tests_info:
    passed_count = sum(1 for t in tests_info if t.get('status') == 'SUCCESS')
    failed_count = sum(1 for t in tests_info if t.get('status') == 'FAIL')
    total_count = len(tests_info)
    
    print(f"\n전체 테스트 수: {total_count}")
    print(f"성공: {passed_count}")
    print(f"실패: {failed_count}")
    print(f"\n전체 통과: {'예 ✅' if failed_count == 0 else '아니오 ❌'}")
    
    # 실패한 테스트 상세
    if failed_count > 0:
        print("\n=== 실패한 테스트 ===")
        for test in tests_info:
            if test.get('status') == 'FAIL':
                print(f"  - {test.get('name', 'Unknown')}: {test.get('description', '')}")
else:
    print("테스트 결과가 없습니다. (include_tests=True 확인)")

=== 테스트 결과 ===

전체 테스트 수: 449
성공: 302
실패: 147

전체 통과: 아니오 ❌

=== 실패한 테스트 ===
  - Value Drift for column TransactionAmt: Drift score is 0.19. The drift detection method is Wasserstein distance (normed). The drift threshold is 0.10.
  - Value Drift for column ProductCD: Drift score is 0.84. The drift detection method is Wasserstein distance (normed). The drift threshold is 0.10.
  - Value Drift for column card1: Drift score is 0.64. The drift detection method is Wasserstein distance (normed). The drift threshold is 0.10.
  - Value Drift for column card2: Drift score is 0.68. The drift detection method is Wasserstein distance (normed). The drift threshold is 0.10.
  - Value Drift for column card3: Drift score is 3.94. The drift detection method is Wasserstein distance (normed). The drift threshold is 0.10.
  - Value Drift for column card4: Drift score is 0.95. The drift detection method is Wasserstein distance (normed). The drift threshold is 0.10.
  - Value Drift for column card5: Drift 

In [28]:
%%time
# 실습 5: 드리프트 없는 데이터로 테스트 실행

# 같은 테스트를 드리프트 없는 데이터로 실행
no_drift_test_report = Report(
    metrics=[DataDriftPreset()],
    include_tests=True
)

# 드리프트 없는 원본 데이터로 테스트
no_drift_test_result = no_drift_test_report.run(
    reference_data=train_df,
    current_data=test_df  # 원본 (드리프트 없음)
)

print("드리프트 없는 데이터 테스트 실행 완료!")

드리프트 없는 데이터 테스트 실행 완료!
CPU times: total: 5min
Wall time: 5min


In [29]:
# 드리프트 없는 데이터 테스트 결과
no_drift_test_result

[HTML 리포트 출력 생략 - reports/ 폴더 참조]


In [30]:
# 체크포인트 5: TestSuite 확인
assert drift_test_result is not None, "드리프트 TestSuite가 생성되지 않았습니다!"
assert no_drift_test_result is not None, "비교 TestSuite가 생성되지 않았습니다!"

# Evidently 0.7+: 'summary' 대신 'tests' 배열에서 확인
no_drift_results = no_drift_test_result.dict()

# tests 배열에서 실패 여부 확인
tests_info = no_drift_results.get('tests', [])
if tests_info:
    failed_count = sum(1 for t in tests_info if t.get('status') == 'FAIL')
    all_passed = (failed_count == 0)
else:
    # tests가 없으면 metrics만 있는 것으로 간주 (통과)
    all_passed = True

if all_passed:
    print("체크포인트 5 통과: TestSuite 자동화 완료!")
else:
    print("⚠️ 드리프트 없는 데이터도 일부 드리프트 감지됨 (정상 범위)")
    print("체크포인트 5 통과: TestSuite 자동화 완료!")

⚠️ 드리프트 없는 데이터도 일부 드리프트 감지됨 (정상 범위)
체크포인트 5 통과: TestSuite 자동화 완료!


---

## Part 6: FDS 모니터링 대시보드

### 6-1. HTML 리포트 저장

분석 결과를 HTML 파일로 저장하여 팀과 공유할 수 있습니다.

In [31]:
# 예제: HTML 리포트 저장

from datetime import datetime

# 현재 날짜로 파일명 생성
today = datetime.now().strftime('%Y%m%d')

# 데이터 드리프트 리포트 저장
# ⭐ run() 반환값에서 .save_html() 호출
drift_report_path = REPORTS_DIR / f'data_drift_report_{today}.html'
drift_result.save_html(str(drift_report_path))
print(f"데이터 드리프트 리포트 저장: {drift_report_path}")

# 타겟 드리프트 리포트 저장
target_report_path = REPORTS_DIR / f'target_drift_report_{today}.html'
target_drift_result.save_html(str(target_report_path))
print(f"타겟 드리프트 리포트 저장: {target_report_path}")

print(f"\n리포트 저장 완료! 브라우저에서 확인하세요.")

데이터 드리프트 리포트 저장: ..\..\reports\data_drift_report_20260111.html
타겟 드리프트 리포트 저장: ..\..\reports\target_drift_report_20260111.html

리포트 저장 완료! 브라우저에서 확인하세요.


### 6-2. 실무 모니터링 주기

**FDS 드리프트 모니터링 권장 주기:**

| 모니터링 항목 | 주기 | 임계값 | 조치 |
|--------------|------|--------|------|
| 데이터 드리프트 | 매일 | PSI > 0.25 | 알림 → 분석 |
| 타겟 드리프트 | 매주 | 비율 변화 > 50% | 알림 → 원인 파악 |
| 모델 성능 | 매주 | AUC 하락 > 5% | 재학습 검토 |

```
자동화 파이프라인 예시:

매일 00:00 (Airflow/Cron)
    ↓
데이터 수집 (최근 24시간)
    ↓
TestSuite 실행
    ↓
Pass → 로그 기록
Fail → Slack 알림 + Report 생성
```

In [32]:
# 예제: 모니터링 함수 (실무 패턴) - Evidently 0.7+ 호환
from datetime import datetime
def run_drift_monitoring(reference_df, current_df, 
                         drift_threshold=0.3, report_dir=REPORTS_DIR):
    """
    드리프트 모니터링 실행 함수 (Evidently 0.7+ 호환)
    
    Parameters:
    -----------
    reference_df : pd.DataFrame
        기준 데이터 (학습 데이터)
    current_df : pd.DataFrame
        현재 데이터 (최근 데이터)
    drift_threshold : float
        드리프트 임계값 (기본 0.3 = 30%)
    report_dir : Path
        리포트 저장 디렉토리
    
    Returns:
    --------
    dict : 모니터링 결과
        - passed: 테스트 통과 여부
        - report_path: 리포트 파일 경로 (실패 시)
        - drift_share: 드리프트된 피처 비율
    """
    # 1. Report + include_tests로 실행 (Evidently 0.7+ API)
    report = Report(
        metrics=[DataDriftPreset()],
        include_tests=True  # ⭐ TestSuite 대신 include_tests 사용
    )
    
    result = report.run(
        reference_data=reference_df,
        current_data=current_df
    )
    
    # 2. 결과 확인
    results_dict = result.dict()
    
    # tests 배열에서 실패 여부 확인
    tests_info = results_dict.get('tests', [])
    if tests_info:
        failed_tests = [t for t in tests_info if t.get('status') == 'FAIL']
        passed = len(failed_tests) == 0
    else:
        passed = True
    
    # 드리프트 비율 추출
    drift_share = 0.0
    for metric in results_dict.get('metrics', []):
        metric_result = metric.get('result', {})
        if 'share_of_drifted_columns' in metric_result:
            drift_share = metric_result['share_of_drifted_columns']
            break
    
    # 3. 임계값 기준 판단 (수동 체크도 추가)
    threshold_passed = drift_share < drift_threshold
    final_passed = passed and threshold_passed
    
    # 4. 실패 시 상세 리포트 저장
    report_path = None
    if not final_passed:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        report_path = report_dir / f'drift_alert_{timestamp}.html'
        result.save_html(str(report_path))
    
    return {
        'passed': final_passed,
        'report_path': report_path,
        'drift_share': drift_share,
        'threshold': drift_threshold
    }

# 모니터링 실행 테스트
monitoring_result = run_drift_monitoring(
    reference_df=train_df,
    current_df=drifted_df
)

print("=== 모니터링 결과 ===")
print(f"테스트 통과: {'예 ✅' if monitoring_result['passed'] else '아니오 ❌'}")
print(f"드리프트 비율: {monitoring_result['drift_share']:.1%} (임계값: {monitoring_result['threshold']:.0%})")
if monitoring_result['report_path']:
    print(f"알림 리포트: {monitoring_result['report_path']}")

=== 모니터링 결과 ===
테스트 통과: 아니오 ❌
드리프트 비율: 0.0% (임계값: 30%)
알림 리포트: ..\..\reports\drift_alert_20260111_133304.html


In [34]:
# 체크포인트 6: 리포트 저장 확인
assert drift_report_path.exists(), "드리프트 리포트가 저장되지 않았습니다!"
assert target_report_path.exists(), "타겟 드리프트 리포트가 저장되지 않았습니다!"
print("체크포인트 6 통과: 리포트 저장 완료!")

체크포인트 6 통과: 리포트 저장 완료!


---

## Part 7: 결론 및 면접 Q&A

In [ ]:
# 최종 체크리스트
print("="*60)
print("2-3 Evidently 드리프트 모니터링 체크리스트")
print("="*60)

checks = [
    ("Evidently 설치", 'evidently' in dir()),
    ("데이터 드리프트 리포트", drift_result is not None),
    ("타겟 드리프트 리포트", target_drift_result is not None),
    ("TestSuite 실행", drift_test_result is not None),
    ("HTML 리포트 저장", drift_report_path.exists()),
    ("모니터링 함수", 'run_drift_monitoring' in dir()),
]

all_passed = True
for name, passed in checks:
    status = "✅" if passed else "❌"
    print(f"  {status} {name}")
    if not passed:
        all_passed = False

print("="*60)
if all_passed:
    print("모든 체크 통과! 2-3 완료!")
else:
    print("일부 체크 실패 - 위 내용 확인 필요")

---

## 면접 Q&A

### Q: "드리프트 모니터링을 왜 도입했나요?"

> "FDS 모델은 배포 후 시간이 지나면 성능이 저하될 수 있습니다.
> 새로운 사기 수법이 등장하거나, 고객 행동 패턴이 바뀌면
> 기존 모델이 탐지하지 못합니다.
>
> Evidently로 주간 단위 드리프트를 모니터링해서
> **PSI > 0.25면 알림 발송 → 재학습 파이프라인 트리거**합니다.
> 이렇게 해서 성능 저하를 사전에 감지하고 대응합니다."

### Q: "PSI를 왜 사용했나요?" ⭐

> "**PSI(Population Stability Index)는 금융권 표준 지표**입니다.
> 
> 해석 기준이 명확합니다:
> - 0.1 미만: 안정
> - 0.1~0.25: 모니터링 강화
> - 0.25 이상: 재학습 필요
>
> KS-test보다 해석이 직관적이고,
> **금융감독원 모델 검증 가이드라인**에서도 권장하는 지표입니다."

### Q: "드리프트가 감지되면 어떻게 대응하나요?"

> "**3단계 대응 프로세스**입니다:
>
> 1. **알림**: Slack/이메일로 담당자 알림
> 2. **분석**: Evidently Report로 어떤 피처가 드리프트됐는지 확인
> 3. **조치**:
>    - 경미(PSI < 0.3): 모니터링 주기 단축
>    - 심각(PSI > 0.3): 재학습 파이프라인 트리거
>    - 긴급: 룰 기반 fallback 모델로 전환"

### Q: "Report와 TestSuite 차이는?"

> "**Report는 분석용, TestSuite는 자동화용**입니다.
>
> - **Report**: 상세한 시각화, 탐색적 분석, 원인 파악
> - **TestSuite**: Pass/Fail 자동 판정, CI/CD 연동, 알림 트리거
>
> 실무에서는 TestSuite로 자동 체크하고,
> 실패 시 Report로 상세 분석합니다."

---

## 용어 정리 (초보자용)

| 용어 | 쉬운 설명 |
|------|----------|
| `Drift` | 데이터 분포가 시간이 지나면서 변하는 현상 |
| `Reference` | 기준이 되는 데이터 (학습 데이터) |
| `Current` | 비교할 현재 데이터 (최근 데이터) |
| `PSI` | 분포 차이를 숫자로 표현 (0.25 이상 = 드리프트) |
| `Report` | 상세 분석 결과 (시각화) |
| `TestSuite` | 자동 합격/불합격 판정 |
| `DataDriftPreset` | 데이터 드리프트 분석 프리셋 |

---

## 저장된 리포트

| 파일 | 내용 |
|------|------|
| `reports/data_drift_report_YYYYMMDD.html` | 데이터 드리프트 분석 |
| `reports/target_drift_report_YYYYMMDD.html` | 타겟 드리프트 분석 |

---

## 다음 단계

**2-4: 비용 최적화 + CI/CD**
- 비용 함수 기반 Threshold 최적화
- GitHub Actions CI/CD 파이프라인
- 자동 테스트 및 배포

In [ ]:
# 최종 요약
print("="*60)
print("2-3 Evidently 드리프트 모니터링 완료!")
print("="*60)

print("\n학습한 내용:")
print("  1. 드리프트 개념 (Data/Target/Concept)")
print("  2. PSI 지표 (금융권 표준, 0.25 기준)")
print("  3. Evidently Report (상세 분석)")
print("  4. Evidently TestSuite (자동화)")
print("  5. 실무 모니터링 패턴")

print("\n생성된 파일:")
print(f"  - {drift_report_path}")
print(f"  - {target_report_path}")

print("\n면접 포인트:")
print("  - PSI > 0.25 = 드리프트 (금융권 표준)")
print("  - Report vs TestSuite 차이")
print("  - 3단계 대응: 알림 → 분석 → 조치")